In [1]:
# Import libraries
import tweepy
import csv
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
from textblob import TextBlob
import re

In [2]:
# load twitter credentials
%run ~\twitter_credentials.py

In [3]:
#Use tweepy.OAuthHandler to create an authentication using the given key and secret
auth = tweepy.OAuthHandler(consumer_key=con_key, consumer_secret=con_secret)
auth.set_access_token(acc_token, acc_secret)

#Connect to the Twitter API using the authentication
api = tweepy.API(auth)

In [4]:
class CustomStreamListener(tweepy.StreamListener):
    # Write each tweet to generic csv file
    def on_status(self, status):
        print (status.text)
        if (not status.retweeted) and ('RT @' not in status.text):
            with open('generic.csv', 'a', encoding='utf-8') as f:
                writer = csv.writer(f, delimiter=',')
                writer.writerow([status.text, status.created_at, len(status.text)])
                print("Written!")
    
    def on_error(self, status_code):
        if status_code == 420:
            return False

In [5]:
my_stream_listener = CustomStreamListener()
my_stream = tweepy.Stream(auth = api.auth, listener=my_stream_listener)

In [6]:
my_stream.filter(track=['#RAW'])

RT @yugiohtas: I fucking love heel Daniel Bryan. #WWE #SDLive
RT @PWU_Armstrong: “You are a small, little man, that I still have to look up at while cutting this promo!” https://t.co/fgPPmypGZZ
RT @WWEAsuka: I will teach you Bushido 🔥
This is what you call the samurai's sense of honour⚔️🤡⚔️ https://t.co/2md09EMtyb
RT @WWENetwork: The #Phenomenal @AJStylesOrg gets his rematch for the #WWEChampionship when he challenges @WWEDanielBryan THIS SUNDAY at #W…
RT @WWE_MandyRose: The real truth is that I was too busy to wait around and had a photoshoot, something @NaomiWWE doesn’t know anything abo…
RT @WWEAsuka: I will teach you Bushido 🔥
This is what you call the samurai's sense of honour⚔️🤡⚔️ https://t.co/2md09EMtyb
RT @WWEAsuka: I will teach you Bushido 🔥
This is what you call the samurai's sense of honour⚔️🤡⚔️ https://t.co/2md09EMtyb
RT @MATTHARDYBRAND: It’s great to see @MustafaAliWWE get the opportunity to face the NEW Dragon of America on #SDLive..

‘Twas a STUPENDOUS…
RT @WWE: Give @WW

KeyboardInterrupt: 

In [7]:
my_stream.disconnect()

In [9]:
# load renamed csv to pandas
raw = pd.read_csv("rawtweets.csv", names = ["Text", "Time", "Length"])
raw

,Text,Time,Length
0,I want it during Raw too 😏😆,0:50,27.0
1,NaN,NaN,NaN
2,#RAW please be good this week! it's in my home...,0:50,75.0
3,NaN,NaN,NaN
4,Its nearly time for #RAW will you make it to t...,0:50,101.0
5,NaN,NaN,NaN
6,Another Monday night #RAW that won't trend on...,0:50,140.0
7,NaN,NaN,NaN
8,Could #RAW be any worse than last week? We sha...,0:50,53.0
9,NaN,NaN,NaN


In [10]:
# Remove all NaN's
raw = raw.dropna()

In [11]:
# Removes numbers and special characters from tweet
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [12]:
sid = SentimentIntensityAnalyzer()

In [13]:
# Use textblob to analyze sentiment
def analyze_sentiment(tweet):
    analysis = TextBlob(clean_tweet(tweet))
    return analysis.sentiment.polarity

In [14]:
raw['Sentiment'] = raw['Text'].apply(analyze_sentiment)

C:\Users\niles\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
# Convert time column to be in elapsed minutes
raw['Time'] = raw['Time'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
raw

C:\Users\niles\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Text,Time,Length,Sentiment
0,I want it during Raw too 😏😆,50,27.0,-0.230769
2,#RAW please be good this week! it's in my home...,50,75.0,0.234615
4,Its nearly time for #RAW will you make it to t...,50,101.0,-0.120513
6,Another Monday night #RAW that won't trend on...,50,140.0,-0.487179
8,Could #RAW be any worse than last week? We sha...,50,53.0,-0.210256
10,Cuántas ganas tenemos de ver #Raw hoy? Qué dic...,50,140.0,-0.230769
12,"Has WWE managed to turn ""Rowdy"" Ronda Rousey i...",50,129.0,-0.230769
14,I've just read #JaxArmy WHAT THE FUCK!? #RAW,50,44.0,-0.315385
16,@CokeIbushi #RAW be like https://t.co/83AkcJDj...,50,52.0,-0.230769
18,Excuse me?? Shield did not fall victim to McI...,50,85.0,-0.076442


In [16]:
# Get an idea of the sentiment of the episode
raw["Sentiment"].mean()

-0.10438496245968415

In [17]:
# Drop text column for R
raw_for_r = raw.drop(columns=['Text'])
raw_for_r

,Time,Length,Sentiment
0,50,27.0,-0.230769
2,50,75.0,0.234615
4,50,101.0,-0.120513
6,50,140.0,-0.487179
8,50,53.0,-0.210256
10,50,140.0,-0.230769
12,50,129.0,-0.230769
14,50,44.0,-0.315385
16,50,52.0,-0.230769
18,50,85.0,-0.076442


In [18]:
# Write df to csv file
raw_for_r.to_csv("raw_for_r.csv")

In [19]:
# Track Smackdown 
my_stream.filter(track=['#SDLive'])

RT @KingNj90: When you’re the hottest thing in all of WWE and you know it. 

@BeckyLynchWWE 
#SDLive 

 https://t.co/wi7ijPxm2S
RT @WWE: EXCLUSIVE: Why didn't @NaomiWWE vs. @WWE_MandyRose happen tonight? #SDLive General Manager @RealPaigeWWE can only think of ONE #AL…
RT @WWEAsuka: I will teach you Bushido 🔥
This is what you call the samurai's sense of honour⚔️🤡⚔️ https://t.co/2md09EMtyb
This is my new #workplace of logo #design. You can hair me for #create a #logo
https://t.co/MQSBE52SJC

Kremlin… https://t.co/eN9F4H3xgK
Written!
RT @RumblingRumors: Kevin Owens and Sami Zayn Return Update

Read Now:
https://t.co/7TstzQO5FS

#WWE #KevinOwens #SamiZayn #WWENetwork #RAW…


KeyboardInterrupt: 

In [20]:
my_stream.disconnect()

In [21]:
# Load smackdown tweets from csv
sd = pd.read_csv("smackdowntweets.csv", names = ["Text", "Time", "Length"])
sd

,Text,Time,Length
0,PUPPIES! #WWE #RAW #SDLive #SmackDown #205Live...,0:49,106.0
1,NaN,NaN,NaN
2,PUPPIES! #WWE #RAW #SDLive #SmackDown #205Live...,0:49,106.0
3,NaN,NaN,NaN
4,PUPPIES! #WWE #RAW #SDLive #SmackDown #205Live...,0:49,106.0
5,NaN,NaN,NaN
6,PUPPIES! #WWE #RAW #SDLive #SmackDown #205Live...,0:49,106.0
7,NaN,NaN,NaN
8,ALMOST TIME FOR THE MAN SHOW \r\n\r\nBecky's s...,0:49,53.0
9,NaN,NaN,NaN


In [22]:
# Remove NaNs
sd = sd.dropna()

In [23]:
# Add sentiment column
sd['Sentiment'] = sd['Text'].apply(analyze_sentiment)
sd

C:\Users\niles\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Text,Time,Length,Sentiment
0,PUPPIES! #WWE #RAW #SDLive #SmackDown #205Live...,0:49,106.0,-0.230769
2,PUPPIES! #WWE #RAW #SDLive #SmackDown #205Live...,0:49,106.0,-0.230769
4,PUPPIES! #WWE #RAW #SDLive #SmackDown #205Live...,0:49,106.0,-0.230769
6,PUPPIES! #WWE #RAW #SDLive #SmackDown #205Live...,0:49,106.0,-0.230769
8,ALMOST TIME FOR THE MAN SHOW \r\n\r\nBecky's s...,0:49,53.0,0.000000
10,PUPPIES! #WWE #RAW #SDLive #SmackDown #205Live...,0:49,106.0,-0.230769
12,PUPPIES! #WWE #RAW #SDLive #SmackDown #205Live...,0:49,106.0,-0.230769
14,Now the show am excited to watch is #SDLive,0:49,43.0,0.375000
16,BOUGHT 2 CASES OF FUNKO POP TO GET LIMITED CHA...,0:49,129.0,0.364286
18,Yo yo yo yo yo...#SDLive in 10 minutes...Austi...,0:49,140.0,0.575000


In [24]:
# Convert time into elapsed minutes
sd['Time'] = sd['Time'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))


C:\Users\niles\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [25]:
# Get an idea of sentiment
sd["Sentiment"].mean()

0.11356783655171368

In [26]:
# Drop text column for loading into R
sd_for_r = sd.drop(columns=['Text'])

In [27]:
# Write to csv
sd_for_r.to_csv("sd_for_r.csv")

In [32]:
# Load tweets from week 2 of Raw
raw2 = pd.read_csv("rawtweets2.csv", names = ["Text", "Time", "Length"])
raw2

,Text,Time,Length
0,RT @HeelWillMahoney: If you're already whining...,0:49,140.0
1,NaN,NaN,NaN
2,RT @RondaRousey: Who’s ready for Monday Night ...,0:49,106.0
3,NaN,NaN,NaN
4,"Annoyed that RR is getting an ""also starring.....",0:49,115.0
5,NaN,NaN,NaN
6,This was actually badass 🔥🔥,0:49,27.0
7,NaN,NaN,NaN
8,RT @DAZN_JPN: ＼#Raw 次回予告（#1333）💣／\r\n\r\nインターコ...,0:49,140.0
9,NaN,NaN,NaN


In [33]:
raw2 = raw2.dropna()

In [34]:
raw2['Sentiment'] = raw2['Text'].apply(analyze_sentiment)
raw2

C:\Users\niles\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Text,Time,Length,Sentiment
0,RT @HeelWillMahoney: If you're already whining...,0:49,140.0,-0.015385
2,RT @RondaRousey: Who’s ready for Monday Night ...,0:49,106.0,-0.015385
4,"Annoyed that RR is getting an ""also starring.....",0:49,115.0,-0.057692
6,This was actually badass 🔥🔥,0:49,27.0,0.000000
8,RT @DAZN_JPN: ＼#Raw 次回予告（#1333）💣／\r\n\r\nインターコ...,0:49,140.0,-0.230769
10,RT @LeoLondon23: Just before #WWE #RAW I will ...,0:49,140.0,0.028419
12,"pro wrestling is something else, guys...",0:50,40.0,0.000000
14,"I have finals tomorrow guys, so for one last t...",0:50,140.0,0.000000
16,RT @RawAnUncut: @RedRoseLaCubana No Panties No...,0:50,105.0,-0.230769
18,RT @SashaBanksWWE: Good morning! \r\nAnother d...,0:50,96.0,0.234615


In [35]:
# Convert time into elapsed minutes
raw2['Time'] = raw2['Time'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

C:\Users\niles\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [36]:
raw2["Sentiment"].mean()

-0.08391722402881356

In [37]:
# Drop text column for loading into R
raw2_for_r = raw2.drop(columns=['Text'])

In [38]:
# Write to csv
raw2_for_r.to_csv("raw2_for_r.csv")

In [47]:
sd2 = pd.read_csv("sdtweets2.csv", names = ["Text", "Time", "Length"])
sd2

,Text,Time,Length
0,@WWE #SDLive!!,1:01,14.0
1,NaN,NaN,NaN
2,Enjoy #SDLive everyone,1:01,22.0
3,NaN,NaN,NaN
4,Time to lay the Smacketh Down! #SDLive,1:01,38.0
5,NaN,NaN,NaN
6,LETS GOOOOO #SDLive Starts Now,1:01,30.0
7,NaN,NaN,NaN
8,Time for #SDLive\r\n\r\nAlso known as #TheMan'...,1:01,47.0
9,NaN,NaN,NaN


In [48]:
sd2 = sd2.dropna()

In [49]:
sd2['Sentiment'] = sd2['Text'].apply(analyze_sentiment)
sd2

C:\Users\niles\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Text,Time,Length,Sentiment
0,@WWE #SDLive!!,1:01,14.0,0.000000
2,Enjoy #SDLive everyone,1:01,22.0,0.400000
4,Time to lay the Smacketh Down! #SDLive,1:01,38.0,-0.155556
6,LETS GOOOOO #SDLive Starts Now,1:01,30.0,0.000000
8,Time for #SDLive\r\n\r\nAlso known as #TheMan'...,1:01,47.0,0.000000
10,RT @FightfulWrestle: Come talk #SDLive with us...,1:01,72.0,0.000000
12,#SDLive https://t.co/hdVgz9THcT,1:01,31.0,0.000000
14,Time for the A show! #SDLive,1:01,28.0,0.000000
16,RT @WWERDream: First time ever match: ☑️\r\n\r...,1:01,140.0,0.275000
18,RT @GrittyUrbanSaga: 🎮 Watch SMACKDOWN With US...,1:01,95.0,0.000000


In [50]:
sd2['Time'] = sd2['Time'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
sd2

C:\Users\niles\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Text,Time,Length,Sentiment
0,@WWE #SDLive!!,61,14.0,0.000000
2,Enjoy #SDLive everyone,61,22.0,0.400000
4,Time to lay the Smacketh Down! #SDLive,61,38.0,-0.155556
6,LETS GOOOOO #SDLive Starts Now,61,30.0,0.000000
8,Time for #SDLive\r\n\r\nAlso known as #TheMan'...,61,47.0,0.000000
10,RT @FightfulWrestle: Come talk #SDLive with us...,61,72.0,0.000000
12,#SDLive https://t.co/hdVgz9THcT,61,31.0,0.000000
14,Time for the A show! #SDLive,61,28.0,0.000000
16,RT @WWERDream: First time ever match: ☑️\r\n\r...,61,140.0,0.275000
18,RT @GrittyUrbanSaga: 🎮 Watch SMACKDOWN With US...,61,95.0,0.000000


In [51]:
sd2["Sentiment"].mean()

0.10257396490530188

In [52]:
# Drop text column for loading into R
sd2_for_r = sd2.drop(columns=['Text'])

In [53]:
# Write to csv
sd2_for_r.to_csv("sd2_for_r.csv")